<a href="https://colab.research.google.com/github/anadiedrichs/2023-nlp-tps/blob/main/3b_Custom_embedding_con_Gensim_ingredientes_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



CAMBIAR Y USAR EL DATASET USANDO LOS INGREDIENTES

https://www.kaggle.com/datasets/kaggle/recipe-ingredients-dataset



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

['dickinson.txt',
 'bob-dylan.txt',
 'Kanye_West.txt',
 'dj-khaled.txt',
 'patti-smith.txt',
 'radiohead.txt',
 'adele.txt',
 'dr-seuss.txt',
 'kanye-west.txt',
 'britney-spears.txt',
 'nirvana.txt',
 'kanye.txt',
 'leonard-cohen.txt',
 'disney.txt',
 'eminem.txt',
 'blink-182.txt',
 'bruno-mars.txt',
 'bieber.txt',
 'bruce-springsteen.txt',
 'alicia-keys.txt',
 'missy-elliott.txt',
 'al-green.txt',
 'lin-manuel-miranda.txt',
 'drake.txt',
 'amy-winehouse.txt',
 'janisjoplin.txt',
 'nicki-minaj.txt',
 'paul-simon.txt',
 'notorious_big.txt',
 'dolly-parton.txt',
 'beatles.txt',
 'nickelback.txt',
 'lady-gaga.txt',
 'joni-mitchell.txt',
 'rihanna.txt',
 'cake.txt',
 'ludacris.txt',
 'notorious-big.txt',
 'lil-wayne.txt',
 'r-kelly.txt',
 'lorde.txt',
 'michael-jackson.txt',
 'nursery_rhymes.txt',
 'bob-marley.txt',
 'bjork.txt',
 'prince.txt',
 'johnny-cash.txt',
 'jimi-hendrix.txt',
 'Lil_Wayne.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

<ipython-input-24-91c4bad52ba1>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar el modelo generador

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.15625
Loss after epoch 1: 65953.953125
Loss after epoch 2: 65934.265625
Loss after epoch 3: 65711.96875
Loss after epoch 4: 63872.5
Loss after epoch 5: 64142.46875
Loss after epoch 6: 64058.8125
Loss after epoch 7: 64755.4375
Loss after epoch 8: 62585.5625
Loss after epoch 9: 60415.25
Loss after epoch 10: 59823.625
Loss after epoch 11: 58930.1875
Loss after epoch 12: 57726.4375
Loss after epoch 13: 56493.125
Loss after epoch 14: 55842.5625
Loss after epoch 15: 55862.5
Loss after epoch 16: 51695.8125
Loss after epoch 17: 49815.5
Loss after epoch 18: 49568.125
Loss after epoch 19: 48959.75


(156986, 287740)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087147116661072),
 ('sleep', 0.8649678230285645),
 ('help', 0.8594305515289307),
 ('try', 0.8456019759178162),
 ('cry', 0.8387227654457092),
 ('little', 0.8273633718490601),
 ('not', 0.8181102871894836),
 ('seems', 0.8175203204154968),
 ('twist', 0.8154979348182678),
 ('peace', 0.812263011932373)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.21786975860595703),
 ('our', -0.24495810270309448),
 ('come', -0.2613270878791809),
 ('on', -0.26722386479377747),
 ('bang', -0.27329546213150024),
 ('five', -0.2741836607456207),
 ('six', -0.2774038314819336),
 ('baby', -0.2776353061199188),
 ('work', -0.2790372967720032),
 ('four', -0.2792743146419525)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9786694049835205),
 ('three', 0.9761051535606384),
 ('six', 0.966891348361969),
 ('two', 0.9565720558166504),
 ('seven', 0.954232931137085),
 ('sixty', 0.9003373980522156),
 ('one', 0.816501259803772),
 ('us', 0.7762899398803711),
 ('crying', 0.772737979888916),
 ('strawberry', 0.7672452330589294)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409592747688293),
 ('buy', 0.9399508237838745),
 ('much', 0.8996850848197937),
 ('hide', 0.8465201258659363),
 ('just', 0.8415144085884094)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["guitar"])

[('go', -0.513823390007019),
 ('day', -0.5212637782096863),
 ('ah', -0.5232623219490051),
 ('yeah', -0.5335099101066589),
 ('yellow', -0.5400408506393433),
 ('girl', -0.5541737079620361),
 ('four', -0.5550034642219543),
 ('good', -0.5555173754692078),
 ('jude', -0.5578638911247253),
 ('hey', -0.5588455200195312)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.